In [7]:
import json
import pandas as pd
from datetime import datetime

In [2]:
# Function to flatten the nested dictionary
def flatten_dict(d, parent_key='', sep='_'):
    items = []
    for k, v in d.items():
        new_key = f'{parent_key}{sep}{k}' if parent_key else k
        if isinstance(v, dict):
            items.extend(flatten_dict(v, new_key, sep=sep).items())
        else:
            items.append((new_key, v))
    return dict(items)


In [3]:
def find_customer_keys(json_data):
    customer_keys = []
    for key, value in json_data.items():  # this iterates through top level keys, but it is sufficient enough 
        if isinstance(value, dict) and any(sub_key.startswith("gcp-") for sub_key in value):
            customer_keys.append(key)
    return customer_keys


In [4]:
def json_to_csv(json_data):
    # Initialize an empty DataFrame
    df = pd.DataFrame()

    # Dynamically find customer-related keys
    customer_keys = find_customer_keys(json_data)

    for key in customer_keys:
        if isinstance(json_data[key], dict):
            for sub_key, value in json_data[key].items():
                if isinstance(value, dict):
                    # Flatten nested dictionaries without including the customer key in column names
                    flattened_data = flatten_dict(value, parent_key=key)
                    for flat_key, flat_value in flattened_data.items():
                        df.at[sub_key, flat_key] = flat_value
                else:
                    df.at[sub_key, key] = value

    df.reset_index(inplace=True)
    
    df.rename(columns={'index': 'customer'}, inplace=True)

    # Fill NaN values with 0, except for the 'Total' row
    df.loc[df['customer'] != 'Total'] = df.loc[df['customer'] != 'Total'].fillna(0)

    # Remove the existing 'Total' row if it exists
    if 'Total' in df['customer'].values:
        df = df[df['customer'] != 'Total']
    
    # Calculate the sum for each column to create a new 'Total' row
    total_row = df.sum(numeric_only=True)
    total_row['customer'] = 'Total'
    df = df.append(total_row, ignore_index=True)

    # Combine Davion 
    davion_total = df.loc[df['customer'].isin(['gcp-davion-prod_davion-production-usdt', 
                                               'gcp-davion-prod_davion-production']), 
                          df.columns != 'customer'].sum()
    davion_total['customer'] = 'davion-total'
    df = df.append(davion_total, ignore_index=True)
    
    return df


In [5]:
# Replace 'your_file.json' with your JSON file path
with open('example.json', 'r') as json_file:
    json_data = json.load(json_file)

# Convert JSON to DataFrame
df = json_to_csv(json_data)


In [9]:
# Save DataFrame to CSV

# Get the current date
current_date = datetime.now().strftime("%Y-%m-%d")

# Save the aggregated DataFrame to a new CSV, including the date in the filename
csv_filename = f"output_{current_date}.csv"
df.to_csv(aggregate_csv_filename)
print(f"Aggregated data saved to {csv_filename}")



Aggregated data saved to output_2023-11-26.csv


In [6]:
df.head()

,customer,customers_cost,customers_gas_used,customers_proof_cost,customers_onchain_data_cost,n_customers_jobs,customers_execution_resources_n_steps,customers_execution_resources_n_memory_holes,customers_execution_resources_builtin_instance_counter_output_builtin,customers_execution_resources_builtin_instance_counter_pedersen_builtin,customers_execution_resources_builtin_instance_counter_range_check_builtin,customers_execution_resources_builtin_instance_counter_ecdsa_builtin,customers_execution_resources_builtin_instance_counter_bitwise_builtin,customers_execution_resources_builtin_instance_counter_ec_op_builtin,customers_execution_resources_builtin_instance_counter_keccak_builtin,customers_execution_resources_builtin_instance_counter_poseidon_builtin
0,gcp-paradigm-otc-prod_potc-production,19.227919,546.94,3.523052,15.704867,2485.0,1.618660e+10,3.201403e+08,452136.0,7.935415e+07,5.341809e+08,125027.0,0.0,0.0,0.0,0.000000e+00
1,gcp-starknet-production_starknet-mainnet,518.825209,15729.67,50.531383,468.293825,17477.0,1.750104e+11,7.996586e+09,28527481.0,3.620579e+09,4.856607e+09,1377778.0,14713451.0,5374984.0,30.0,2.904832e+09
2,immutable-production_immutable-production,1.503134,43.68,1.126300,0.376835,142.0,1.469648e+09,1.203311e+06,8259.0,1.358454e+08,3.127177e+07,1203169.0,0.0,0.0,0.0,0.000000e+00
3,gcp-myria-prod_myria-production,0.209613,6.03,0.028654,0.180959,43.0,1.157886e+08,1.371804e+06,2039.0,1.689797e+06,5.168909e+06,51387.0,0.0,0.0,0.0,0.000000e+00
4,gcp-davion-prod_davion-production-usdt,0.166367,4.88,0.058207,0.108160,42.0,1.789950e+08,1.190930e+07,3323.0,4.322440e+06,7.180590e+06,38998.0,0.0,0.0,0.0,0.000000e+00


In [57]:
# # if Avishag wants me to aggregate 

# combined_df = pd.concat([df, df_2], ignore_index=True)
# aggregate_df = combined_df.groupby('customer').sum().reset_index()